In [ ]:
!pip install bs4
!pip install requests
!pip install pandas

In [64]:
from  bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [65]:
# Function to extract Product Title
def get_title(soup):
    
    try:
        # Outer Tag Object
        title = soup.find('span', attrs={'id':'productTitle'})
        
        # Inner NaigatableString Object
        title_value = title.text
        
        # Title as a string value
        title_string = title_value.strip()
        
    except AttributeError:
        title_string = ''
        
    return title_string

# Function to extract Product Price
def get_price(soup):
    
    try:
        price = soup.find('span', attrs={'id':'priceblock_ourprice'}).text.strip()

    except AttributeError:
        try:
            # if there is some deal price
            price = soup.find('span', attrs={'id':'priceblock_dealprice'}).text.strip()
        except:
            price = ''
        
    return price

def get_rating(soup):
    
    try:
        rating = soup.find('i', attrs={'class':'a-icon a-icon-star a-star-4-5'}).text.strip()
        
    except AttributeError:
        try:
            rating = soup.find('span', attrs={'id':'priceblock_ourprice'}).text.strip()
        except:
            rating = ''
    
    return rating

def get_review_count(soup):
    
    try:
        review_count = soup.find('span', attrs={'id':'acrCustomerReviewText'}).text.strip()
    except AttributeError :
        review_count = ''
    
    return review_count

def get_availability(soup):
    
    try:
        available = soup.find('div', attrs={'id':'availability'})
        available = available.find('span').string.strip()
    except AttributeError:
        available = 'Not Available'
        
    return available

In [66]:
if __name__ == '__main__':
    
    #add your user agent
    HEADERS = ({'User-Agent':'', 'Accept-Language':'en-US, en;q=0.5'})

    URL = 'https://www.amazon.com/s?k=playstation+5&crid=P73DHBKAPEHT&sprefix=playstation+5%2Caps%2C333&ref=nb_sb_ss_ts-doa-p_1_13'

    webpage = requests.get(URL, headers=HEADERS)

    soup = BeautifulSoup(webpage.content, 'html.parser')

    links = soup.find_all("a", attrs={'class':'a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal'})

    links_list = []

    for link in links:
        links_list.append(link.get('href'))

    d = {'title':[], 'price':[], 'rating':[], 'reviews':[], 'availability':[]}

    for link in links_list:
        new_webpage = requests.get('https://www.amazon.com' + link, headers=HEADERS)

        new_soup = BeautifulSoup(new_webpage.content, 'html.parser')

        d['title'].append(get_title(new_soup))
        d['price'].append(get_price(new_soup))
        d['rating'].append(get_rating(new_soup))
        d['reviews'].append(get_review_count(new_soup))
        d['availability'].append(get_availability(new_soup))

    amazon_df = pd.DataFrame.from_dict(d)
    amazon_df['title'].replace('', np.nan, inplace=True)
    amazon_df = amazon_df.dropna(subset=['title'])
    amazon_df.to_csv('amazon_data.csv', index=False)

In [67]:
amazon_df

,title,price,rating,reviews,availability
0,NBA 2K23 - PlayStation 5,$39.99,4.6 out of 5 stars,169 ratings,In Stock
1,Goat Simulator 3 - PlayStation 5,$20.00,,,
2,PlayStation 5 Console,,4.4 out of 5 stars,55 ratings,Available by invitation
3,PS5 Console – God of War Ragnarök Bundle,,4.5 out of 5 stars,40 ratings,Available by invitation
4,PlayStation 5 Digital Edition,,4.3 out of 5 stars,31 ratings,Available by invitation
5,PS5 Digital Edition- Horizon Forbidden West Bu...,,,128 ratings,Available by invitation
6,PS5 Digital Edition – God of War Ragnarök Bundle,,,8 ratings,Available by invitation
7,PS5 Console- Horizon Forbidden West Bundle,,,"5,287 ratings",Available by invitation
8,God of War Ragnarök Launch Edition - PlayStati...,$69.00,4.6 out of 5 stars,120 ratings,In Stock
9,PlayStation DualSense Wireless Controller - St...,$49.99,,"63,995 ratings",In Stock
